# Webscraper energieprijzen
URLs:
* https://www.epexspot.com/en/market-data
* https://www.energyzero.nl/energiecontract/stroom-en-gas/flextarieven

In [207]:
import pandas as pd
import requests
import webbrowser
from pathlib import Path
from datetime import date, timedelta
import lxml.html as html
import dropbox

In [208]:
# We willen de prijzen van morgen
today = date.today()
tomorrow = today + timedelta(1)

In [209]:
# Browse naar website
url = "https://www.epexspot.com/en/market-data"
params = {
    "market_area": "NL",
    "trading_date": today,
    "delivery_date": tomorrow,
    "modality": "Auction",
    "sub_modality": "DayAhead",
    "product": 60,
    "data_mode": "table",
    
}
page = requests.get(url, params=params)
assert page.status_code == 200

# Get html content
content = html.fromstring(page.content)

In [210]:
# Selecteer <table>
tbody = content.cssselect("tbody")[0]
tr_s = tbody.cssselect("tr")
assert len(tr_s) == 24

In [200]:
# Optioneel: bekijk html in browser
path = Path().resolve()
filename = "tmp.html"
with open(path / filename, "w") as f:
    f.write(str(page.content))

webbrowser.open_new_tab(f"file:///{path / filename}")

True

In [211]:
# Scrape de tabel
d = {
    "day": tomorrow,
    "hours": sorted(range(24)),
    "buy_volume": [],
    "sell_volume": [],
    "volume": [],
    "price": [],
}
column_headers = ["buy_volume", "sell_volume", "volume", "price"]
for tr in tr_s:
    for column_header, td in zip(column_headers, tr.cssselect("td")):
        d[column_header].append(td.text)

df = pd.DataFrame(d)
df

,day,hours,buy_volume,sell_volume,volume,price
0,2021-04-06,0,"2,659.8","3,481.4","3,481.4",18.84
1,2021-04-06,1,"2,573.9","3,657.6","3,657.6",17.35
2,2021-04-06,2,"2,542.3","3,752.5","3,752.5",15.76
3,2021-04-06,3,"2,554.8","3,743.4","3,743.4",17.02
4,2021-04-06,4,"2,624.7","3,584.7","3,584.7",17.68
5,2021-04-06,5,"2,709.5","3,610.0","3,610.0",30.23
6,2021-04-06,6,"2,504.5","3,648.9","3,648.9",53.69
7,2021-04-06,7,"3,027.8","3,241.2","3,241.2",67.81
8,2021-04-06,8,"3,596.9","3,434.1","3,596.9",94.74
9,2021-04-06,9,"3,683.1","3,573.2","3,683.1",60.00


In [206]:
# Connect to Dropbox
access_token = "sl.AuU7qW1KakbdUtUFZO079vPc9jXdRyK6VcCG-uZrVZSmrY2gG_rf_TkdMHjvYhIC1IperO7Mif3XP1AZvHmWFbBMeUyDYWwy0AqgyhxAsooODs7-aMK2slXwG4COuU6uU7xVSfQ"
dbx = dropbox.Dropbox(access_token)
assert dbx.users_get_current_account().name.abbreviated_name == "AB"

# Upload csv
binary_csv = df.to_csv(index=False).encode('utf-8')
filename = f"/epexspot_{tomorrow}.csv"
dbx.files_upload(binary_csv, filename, mode=dropbox.files.WriteMode("overwrite"))

FileMetadata(client_modified=datetime.datetime(2021, 4, 5, 13, 8, 18), content_hash='015ef2ab1309b9bfc9cb5ebda852ad4c5e3fabd21b14a8ee7f263c89c7218949', export_info=NOT_SET, file_lock_info=NOT_SET, has_explicit_shared_members=NOT_SET, id='id:jwa9lCFnHysAAAAAAAAKtw', is_downloadable=True, media_info=NOT_SET, name='epexspot_2021-04-05.csv', parent_shared_folder_id=NOT_SET, path_display='/epexspot_2021-04-05.csv', path_lower='/epexspot_2021-04-05.csv', property_groups=NOT_SET, rev='015bf39643b4c6e000000022fc32290', server_modified=datetime.datetime(2021, 4, 5, 13, 8, 19), sharing_info=NOT_SET, size=1236, symlink_info=NOT_SET)